In [1]:
# !pip install praw

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from kafka import KafkaProducer
import praw
import json

def get_submissions(rdd, seen_ids):
    # Set up Reddit API
    reddit = praw.Reddit(client_id='r_mi6b5G4SkVR-uWyp37kw',
                         client_secret='c0-p81DnRU9CB9Weq70ahiTFMqeb8g',
                         user_agent='5003-test')
    subreddit = reddit.subreddit('askreddit')
    # 'doesanybodyelse' very good subreddit but frequency is low
    # 'askreddit' extremely frequent but may not be relevant
    new_python = subreddit.new(limit=5)

    # Filter out duplicate submissions based on their IDs
    new_submissions = [submission for submission in new_python if submission.id not in seen_ids]
    seen_ids.update(submission.id for submission in new_submissions)

    # Convert the submission titles to a list and return as a new RDD
    return rdd.context.parallelize([{"id": submission.id, "title": submission.title} for submission in new_submissions])

def send_to_kafka(rdd):
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    
    for record in rdd.collect():
        producer.send('reddit_titles', json.dumps(record).encode('utf-8'))

    producer.flush()
    producer.close()

if __name__ == "__main__":
    # Create a SparkContext or get an existing one if it's already running
    sc = SparkContext.getOrCreate()
    ssc = StreamingContext(sc, 30)  # Streaming context with batch interval of 15 seconds
    submissions = ssc.socketTextStream("localhost", 9999)

    # Keep track of the IDs of seen submissions
    seen_ids = set()

    # Apply the transformation function to create a new DStream of submission titles
    submission_titles = submissions.transform(lambda rdd: get_submissions(rdd, seen_ids))

    # Send data to Kafka
    submission_titles.foreachRDD(send_to_kafka)

    # Print the submission titles every 15 seconds
    submission_titles.pprint()

    ssc.start()
    ssc.awaitTermination()


-------------------------------------------
Time: 2023-04-20 16:50:30
-------------------------------------------
{'id': '12srp7a', 'title': 'Does she like me and also how do I tell her?'}
{'id': '12srncw', 'title': 'Todays gonna be terrible •~•'}
{'id': '12srncj', 'title': 'Todays gonna be terrible •~•'}
{'id': '12srlul', 'title': 'My body count is 4'}
{'id': '12srluf', 'title': 'give me your problems i will give you pro tip.'}

